Shravani Isukapalli
PES2UG23CS552

In [1]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.7/111.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 137.5/137.5 kB 6.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 500.5/500.5 kB 22.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.1/158.1 kB 10.1 MB/s eta 0:00:00


In [3]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.0)

Enter your Groq API Key: ··········


In [6]:
question = "A student buys 4 notebooks for $8 each. He gets a discount of $6. What is the final amount he pays?"
prompt_standard = f"Answer this question: {question}"
print("--- STANDARD (Llama3.1-8b) ---")
print(llm.invoke(prompt_standard).content)

--- STANDARD (Llama3.1-8b) ---
To find the final amount the student pays, we need to calculate the total cost of the notebooks and then subtract the discount.

The total cost of the notebooks is:
4 notebooks * $8 each = $32

The student gets a discount of $6, so we subtract this from the total cost:
$32 - $6 = $26

The final amount the student pays is $26.


In [7]:

prompt_cot = f"Answer this question. Let's think step by step. {question}"

print("--- Chain of Thought (Llama3.1-8b) ---")
print(llm.invoke(prompt_cot).content)

--- Chain of Thought (Llama3.1-8b) ---
To find the final amount the student pays, we need to follow these steps:

1. Calculate the total cost of the notebooks before the discount:
   Number of notebooks = 4
   Cost per notebook = $8
   Total cost = Number of notebooks * Cost per notebook
   Total cost = 4 * $8
   Total cost = $32

2. Apply the discount:
   Discount = $6
   Final amount = Total cost - Discount
   Final amount = $32 - $6
   Final amount = $26

So, the final amount the student pays is $26.


TOT and GOT

In [8]:
%pip install python-dotenv --upgrade --quiet langchain langchain-groq


In [9]:
from dotenv import load_dotenv
load_dotenv()

import getpass
import os
from langchain_groq import ChatGroq

if "GROQ_API_KEY" not in os.environ:
    os.environ["GROQ_API_KEY"] = getpass.getpass("Enter your Groq API Key: ")

# Using Llama3.1-8b
llm = ChatGroq(model="llama-3.1-8b-instant", temperature=0.7) # Creativity needed

In [10]:
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.runnables import RunnableParallel, RunnableLambda
from langchain_core.output_parsers import StrOutputParser

problem = "How can an engineering student improve productivity while preparing for exams?"

prompt_branch = ChatPromptTemplate.from_template(
    "Problem: {problem}. Give one unique and practical solution. Solution {id}:"
)

branches = RunnableParallel(
    sol1=prompt_branch.partial(id="1") | llm | StrOutputParser(),
    sol2=prompt_branch.partial(id="2") | llm | StrOutputParser(),
    sol3=prompt_branch.partial(id="3") | llm | StrOutputParser(),
)

prompt_judge = ChatPromptTemplate.from_template(
"""
I have three proposed solutions for the problem: "{problem}"

Solution 1:
{sol1}

Solution 2:
{sol2}

Solution 3:
{sol3}

Act as an Academic Productivity Expert. Select the most effective and sustainable solution for long-term productivity and explain why.
"""
)

tot_chain = (
    RunnableParallel(problem=RunnableLambda(lambda x: x), branches=branches)
    | (lambda x: {**x["branches"], "problem": x["problem"]})
    | prompt_judge
    | llm
    | StrOutputParser()
)

print("--- Tree of Thoughts (ToT) Result ---")
print(tot_chain.invoke(problem))


--- Tree of Thoughts (ToT) Result ---
After reviewing the three proposed solutions, I recommend **Solution 1: Implement the "Pomodoro Technique with a Twist"** as the most effective and sustainable solution for long-term productivity. Here's why:

1. **Combination of techniques**: This solution combines the traditional Pomodoro Technique with active recall, spaced repetition, and the Stoplight System. This combination provides a comprehensive approach to learning and retention, making it more effective than relying on a single technique.
2. **Personalization**: The Stoplight System allows students to categorize their study material based on their needs, making it a flexible and adaptable solution. This system helps students identify areas where they need to focus their efforts, ensuring that they prioritize their time and energy effectively.
3. **Motivation and engagement**: The use of a "twist" on the traditional Pomodoro Technique makes it more engaging and motivating for students. T

In [11]:

prompt_draft = ChatPromptTemplate.from_template(
    "Write a 1-sentence idea about {topic} from the perspective of {perspective}."
)

drafts = RunnableParallel(
    draft_ai=prompt_draft.partial(perspective="Artificial Intelligence") | llm | StrOutputParser(),
    draft_iot=prompt_draft.partial(perspective="Internet of Things") | llm | StrOutputParser(),
    draft_cloud=prompt_draft.partial(perspective="Cloud Computing") | llm | StrOutputParser(),
)

prompt_combine = ChatPromptTemplate.from_template(
"""
I have three ideas about "{topic}":

1. Artificial Intelligence: {draft_ai}

2. Internet of Things: {draft_iot}

3. Cloud Computing: {draft_cloud}

Combine these ideas into one unified system. Explain how all three technologies work together to create an intelligent and efficient solution.
Write one paragraph.
"""
)

got_chain = (
    RunnableParallel(topic=RunnableLambda(lambda x: x), drafts=drafts)
    | (lambda x: {**x["drafts"], "topic": x["topic"]})
    | prompt_combine
    | llm
    | StrOutputParser()
)

print("--- Graph of Thoughts (GoT) Result ---")
print(got_chain.invoke("Smart Cities"))


--- Graph of Thoughts (GoT) Result ---
In a unified Smart City system, Artificial Intelligence (AI) serves as the strategic decision-maker, leveraging the vast amounts of data generated by the Internet of Things (IoT) to optimize urban systems. This data is collected and analyzed through a Cloud Computing platform, which provides real-time insights and efficient resource allocation. The AI then uses these insights to make data-driven decisions, streamlining transportation, waste management, and public services while maximizing energy efficiency. Additionally, the AI-driven platform enables seamless citizen engagement, providing personalized services, and enhancing overall quality of life through the IoT's ubiquitous sensing capabilities. By integrating these technologies, the Smart City becomes a harmonious blend of human experience and technological innovation, where residents, businesses, and infrastructure thrive in a connected, efficient, and sustainable ecosystem.
